# Ví dụ Sử Dụng Công Cụ AutoGen


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Tạo Client

Trong ví dụ này, chúng ta sẽ sử dụng [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) để truy cập vào LLM.

`model` được định nghĩa là `gpt-4o-mini`. Hãy thử thay đổi model sang một model khác có sẵn trên marketplace của GitHub Models để xem các kết quả khác nhau.

Để kiểm tra nhanh, chúng ta sẽ chạy một prompt đơn giản - `Thủ đô của Pháp là gì`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Định nghĩa các hàm

Trong ví dụ này, chúng ta sẽ cung cấp cho tác nhân quyền truy cập vào một công cụ là một hàm với danh sách các điểm đến du lịch có sẵn và tình trạng khả dụng của chúng.

Bạn có thể hình dung đây là một tình huống mà một đại lý du lịch có thể truy cập vào cơ sở dữ liệu du lịch, chẳng hạn.

Khi bạn thực hiện ví dụ này, hãy thoải mái thử định nghĩa các hàm và công cụ mới mà tác nhân có thể gọi.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Định nghĩa Công cụ Hàm  
Để agent có thể sử dụng `vacation_destinations` như một `FunctionTool`, chúng ta cần định nghĩa nó như vậy.  

Chúng ta cũng sẽ cung cấp một mô tả về công cụ này, điều này giúp agent xác định công cụ đó được sử dụng để làm gì liên quan đến nhiệm vụ mà người dùng yêu cầu.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Định nghĩa agent

Bây giờ chúng ta có thể tạo agent trong đoạn mã dưới đây. Chúng ta định nghĩa `system_message` để cung cấp hướng dẫn cho agent về cách giúp người dùng tìm kiếm điểm đến du lịch.

Chúng ta cũng đặt tham số `reflect_on_tool_use` thành true. Điều này cho phép sử dụng LLM để lấy phản hồi từ việc gọi công cụ và gửi phản hồi bằng ngôn ngữ tự nhiên.

Bạn có thể đặt tham số này thành false để thấy sự khác biệt.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Chạy Agent

Bây giờ chúng ta có thể chạy agent với tin nhắn ban đầu từ người dùng yêu cầu thực hiện một chuyến đi đến Tokyo.

Bạn có thể thay đổi điểm đến thành phố này để xem cách agent phản hồi về tính khả dụng của thành phố.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
